This Time with timeseries

In [1]:
using JuMP
using Clp
using DataFrames
using LinearAlgebra
using CSV

In [24]:
L_df = CSV.read("lines.csv")
N_df = CSV.read("nodes.csv")
P_df = CSV.read("plants.csv")
D_df = CSV.read("demand.csv")

,1,2,3,4,5,6,7,8,9,10,11,12
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,150,150,150,150,150,150,150,150,150,150,150,150
2,150,150,150,150,150,150,150,150,150,150,150,150


In [30]:
b = vec(Array(select(L_df,"b")))
A = zeros(nrow(L_df),nrow(N_df))
NODES = vec(Array(select(N_df,"index")))
SLACK = 14
PLANTS = vec(Array(select(P_df,"index")))
LINES = vec(Array(select(L_df,"index")))
T = [1,2]

demand= D_df
#Demand = Dict()
#Demand["t1"]=Array(D_df[1,:])
#Demand["t2"]=Array(D_df[2,:])
l_max = vec(Array(select(L_df,"lmax")))


#calculation of incidence matrix A
counter= 1
for i in eachrow(L_df)
    wert1 = L_df[counter,:][2]
    wert2 = L_df[counter,:][3]
    A[counter,wert1] =1
    A[counter,wert2] =-1
    global counter +=1
end


In [27]:
#Account for double entries
#plants_at_node dict
#gmax dict
#mc
plants_at_node = Dict()
gmax = Dict()
mc = Dict()

for i in eachrow(P_df)
    if !haskey(plants_at_node,i["node"])
        plants_at_node[i["node"]]=[i["index"]]
    else
        push!(plants_at_node[i["node"]],i["index"])
    end
    gmax[i["index"]] = i["g_max"]
    mc[i["index"]] = i["mc"]
    
    
    
end
plants_at_node[SLACK]=[]
plants_at_node
mc

Dict{Any,Any} with 16 entries:
  "p17" => 0
  "p3"  => 0
  "p12" => 6
  "p15" => 5
  "p13" => 0
  "p16" => 5
  "p4"  => 8
  "p5"  => 0
  "p10" => 15
  "p7"  => 6
  "p6"  => 0
  "p9"  => 8
  "p2"  => 15
  "p8"  => 8
  "p1"  => 0
  "p11" => 0

In [28]:
#calculate ptdf matrix source:lecture
function calculate_ptdf(A, b_vector, NODES, SLACK)
    """ Create PTDF Based on:
    Topology (A, incedence Matrix): LxN Matrix (1, -1) for nodes connected by line
    Susceptance Vector (B): Linesusceptance, calculated by line paramters and propotional to length
    NODES: Array of nodes
    SLACK: Reference Node to create the inversse of A.T * Bd A

    From documentation: PTDF = (Bd*A)(A.T * Bd * A)^-1
    Bd: Suceptance Vector (for each line) on Diagonal Matrix (LxL)
    Bl: Line Susceptance Matrix Bd*A
    Bn: Nodes Susceptance Matrix (A'*B)*A
    B_inv = Bn^-1 (without slack, otherwise singular)
    PTDF = Bl * B_inv
    """
    ## Calculation based on indices rather than Node Names (of type Str or Symb)
    idx_nodes = collect(1:length(NODES))
    idx_slack = findfirst(x -> x==SLACK, NODES)
    #A = incedence
    Bd = Diagonal(b_vector)
    Bl = Bd*A # Line suceptance Matrix
    Bn = (A'*Bd)*A # Nodes suceptance Matrix
    
    B_inv = zeros(length(NODES),length(NODES))
    B_inv[setdiff(idx_nodes,idx_slack), setdiff(idx_nodes,idx_slack)] = inv(Bn[setdiff(idx_nodes,idx_slack), setdiff(idx_nodes,idx_slack)])
    PTDF = Bl*B_inv
    return PTDF
end


calculate_ptdf (generic function with 1 method)

In [29]:
ptdf = calculate_ptdf(A,b,NODES,SLACK)

26×16 Array{Float64,2}:
  0.395689   -0.0208109  -0.0208109  …  0.0   0.00476269   0.0143332
  0.208622    0.0416217   0.0416217     0.0  -0.00952538  -0.0286665
  0.0         1.0         0.0           0.0   0.0          0.0
  0.395689   -0.0208109  -0.0208109     0.0   0.00476269   0.0143332
  0.395689   -0.0208109  -0.0208109     0.0   0.00476269   0.0143332
 -0.0748266   0.024973    0.024973   …  0.0  -0.00571523  -0.0171999
  0.133796    0.0665948   0.0665948     0.0  -0.0152406   -0.0458664
 -0.0101633   0.0313622   0.0313622     0.0  -0.01561     -0.0496577
 -0.157586   -0.197557   -0.197557      0.0   0.0367796    0.108008
  0.0         0.0         0.0           0.0   0.0          0.0
  0.123632    0.097957    0.097957   …  0.0  -0.0308506   -0.0955241
  0.0542806   0.0368759   0.0368759     0.0   0.0252909    0.0868315
  0.16775     0.166195    0.166195      0.0  -0.0211697   -0.0583503
 -0.208891   -0.215355   -0.215355      0.0  -0.0395905   -0.0746634
 -0.499727   -0.520493 

In [34]:
#ptdf_dict = Dict((l, n) => ptdf[l_idx, n_idx] for (n_idx, n) in enumerate(NODES),
 #                                                 (l_idx, l) in enumerate(LINES))

Dict{Tuple{Int64,Int64},Float64} with 416 entries:
  (23, 8)  => -0.160756
  (6, 9)   => 0.00557968
  (18, 3)  => 0.0463307
  (21, 13) => -1.0
  (22, 16) => -0.562724
  (22, 14) => 0.0
  (23, 10) => -0.160756
  (13, 9)  => 0.0340663
  (26, 4)  => -0.019241
  (8, 9)   => 0.0085404
  (9, 1)   => -0.157586
  (17, 14) => 0.0
  (16, 6)  => -0.0628329
  (16, 12) => -0.0889926
  (16, 13) => 1.66533e-16
  (22, 2)  => -0.601022
  (24, 3)  => -0.0910465
  (7, 16)  => -0.0458664
  (1, 12)  => 0.0350101
  (4, 5)   => 0.228688
  (11, 8)  => 0.0029724
  (19, 4)  => 0.0652165
  (20, 13) => -8.88178e-17
  (15, 4)  => -0.51011
  (23, 16) => -0.255165
  ⋮        => ⋮

In [37]:
##Model and stuff
Fax = Model(Clp.Optimizer)
@variables Fax begin
    G[PLANTS,T] >= 0
    INJ[NODES,T]
end

@objective(Fax, Min, sum(mc[plant] * G[plant,timestep] for plant in PLANTS, timestep in T));

@constraint(Fax, Max_Generation[plant=PLANTS,timestep = T],
            G[plant,timestep] <= gmax[plant]);

@constraint(Fax, EnergyBalance[node=NODES,timestep = T],
            sum(G[plant,timestep] for plant in plants_at_node[node])
            - demand[timestep][node] == INJ[node,timestep]);

@constraint(Fax, LineMax[line=LINES],
            sum(ptdf[line, node] * INJ[node,timestep] for node in NODES) <= l_max[line]
            );
@constraint(Fax, LineMin[line=LINES],
            sum(ptdf[line, node] * INJ[node,timestep] for node in NODES) >= -l_max[line]
            );

@constraint(Fax, sum(INJ[node,timestep] for node in NODES) == 0);

JuMP.optimize!(Fax)
JuMP.objective_value(Fax)
'
generation_at_node = [length(plants_at_node[node]) > 0 ? sum(value(G[plant]) for plant in plants_at_node[node]) : 0 for node in NODES]
gen = DataFrame(Nodes=NODES,
                Generation=generation_at_node,
                Injections=value.(INJ).data,
                Price=dual.(EnergyBalance).data,
                )
println(gen)
flow = ptdf * value.(INJ).data
println(flow)'

LoadError: BoundsError: attempt to access 2-element Array{Int64,1} at index [3]